In [1]:
import pandas as pd

MAX_TEXT_LENGTH=512

def auto_truncate(val):
    """Truncate the given text."""
    return val[:MAX_TEXT_LENGTH]

# Load Product data and truncate long text fields
all_prods_df = pd.read_csv("product_data.csv", converters={
    'bullet_point': auto_truncate,
    'item_keywords': auto_truncate,
    'item_name': auto_truncate
})

In [2]:
# Contruct a primary key from item ID and domain name
all_prods_df['primary_key'] = (
    all_prods_df['item_id'] + '-' + all_prods_df['domain_name']
)
# Replace empty strings with None and drop
all_prods_df['item_keywords'].replace('', None, inplace=True)
all_prods_df.dropna(inplace=True)
# all_prods_df

# Reset pandas dataframe index
all_prods_df.reset_index(drop=True, inplace=True)

all_prods_df.head()

/tmp/ipykernel_177101/1781557686.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_prods_df['item_keywords'].replace('', None, inplace=True)


,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type,primary_key
0,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE,B07T2JY31Y-amazon.in
1,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE,B0854774X5-amazon.in
2,B085494J1X,Amazon,IN,71iB+TUHVyL,amazon.in,"Snug fit for Lenovo Z6 Pro, with perfect cut-o...",Back Cover Designer Case Designer Panda Textur...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Panda Texture U...,Lenovo Z6 Pro,UV10833-SL40408,CELLULAR_PHONE_CASE,B085494J1X-amazon.in
3,B08511FPLP,Amazon,IN,81Gd4qnyogL,amazon.in,"Snug fit for Samsung Galaxy M31, with perfect ...",Samsung Galaxy M31 mobile case Samsung Galaxy ...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Old Stambh 3D P...,Samsung Galaxy M31,gz8759-SL40523,CELLULAR_PHONE_CASE,B08511FPLP-amazon.in
4,B08542LQNW,Amazon,IN,719JbIZ9gqL,amazon.in,"Snug fit for Huawei Y9 (2019), with perfect cu...",Back Cover Designer Case Designer Happy Mobile...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Happy UV Printe...,Huawei Y9 (2019),UV10721-SL13243,CELLULAR_PHONE_CASE,B08542LQNW-amazon.in


In [3]:
# Num products to use (subset)
NUMBER_PRODUCTS = 2500  

# Get the first 1000 products with non-empty item keywords
product_metadata = ( 
    all_prods_df
     .head(NUMBER_PRODUCTS)
     .to_dict(orient='index')
)

In [4]:
# Check one of the products
product_metadata[0]

{'item_id': 'B07T2JY31Y',
 'marketplace': 'Amazon',
 'country': 'IN',
 'main_image_id': '71vX7qIEAIL',
 'domain_name': 'amazon.in',
 'bullet_point': '3D Printed Hard Back Case Mobile Cover for Sony Xperia Z1 L39H Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty',
 'item_keywords': 'mobile cover back cover mobile case phone case mobile panel phone panel LG mobile case LG phone cover LG back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone cover Sony Xperia back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone 

In [7]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.redis import Redis as RedisVectorStore
from langchain_core.callbacks.manager import CallbackManager

# data that will be embedded and converted to vectors
texts = [
    v['item_name'] for k, v in product_metadata.items()
]

# product metadata that we'll store along our vectors
metadatas = list(product_metadata.values())

# we will use OpenAI as our embeddings provider
embedding = OpenAIEmbeddings(openai_api_key="open-key", model="text-embedding-3-small")

# name of the Redis search index to create
index_name = "products"

# assumes you have a redis stack server running on within your docker compose network
redis_url = "redis://redis:6379"

# create and load redis with documents
vectorstore = RedisVectorStore.from_texts(
    texts=texts,
    metadatas=metadatas,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

ConnectionError: Error -3 connecting to redis:6379. Temporary failure in name resolution.